# Challenge 0

In [2]:
import sys
sys.version

'3.6.5 (default, Jun 17 2018, 12:15:43) \n[GCC 4.2.1 Compatible Apple LLVM 8.0.0 (clang-800.0.42.1)]'

In [4]:
! pip freeze
! pip --version

certifi==2018.8.24
pip 18.0 from /Users/ibrahimmuhammad/miniconda3/envs/py35/lib/python3.5/site-packages/pip (python 3.5)
